## Day 18: Snailfish

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.org/github/mazharenko/AoC-2021/tree/HEAD/notebooks/day18/puzzle.ipynb)

In [ ]:
type SnailNumber =
    | Literal of int ref
    | Pair of (SnailNumber ref * SnailNumber ref)

In [ ]:
#r "nuget: Farkle"

open Farkle 
open Farkle.Builder

let private number = Terminals.genericSigned<int> "Number"

let snailNumber = nonterminal "SnailNumber"
snailNumber.SetProductions (
    !@ number => (ref >> Literal),
    !& "[" .>>. snailNumber .>> "," .>>. snailNumber .>> "]" => (fun a b -> Pair (ref a, ref b))
)

let private snailNumberRuntime = RuntimeFarkle.build snailNumber

let parse s =
    match RuntimeFarkle.parseString snailNumberRuntime s with
    | Ok x -> x
    | Error e -> e |> string |> failwith
    


Installed Packages Farkle, 6.3.2

In [ ]:

type ReductionState = { Depth : int; Exploded: (int*int) option; LiteralToLeft: SnailNumber option; LiteralToRight: SnailNumber option}

let rec explode (numberRef) state =
    match numberRef with
    | {contents = Literal value} ->
        match state.Exploded, state.LiteralToRight with
        | None, _ -> {state with LiteralToLeft = Some numberRef.Value}
        | Some (_, rightValue), None ->
            value.Value <- value.Value + rightValue
            {state with LiteralToRight = Some numberRef.Value}
        | _ -> state 
    | {contents = Pair (leftRef, rightRef)} ->
        match leftRef, rightRef with
        | {contents = Literal leftValue}, {contents = Literal rightValue}
            when (state.Depth >= 4 && state.Exploded.IsNone) ->
                match state.LiteralToLeft with
                | Some (Literal toLeft) -> toLeft.Value <-  toLeft.Value + leftValue.Value
                | None -> ()
                numberRef.Value <- Literal (ref 0)
                {state with Exploded = Some (leftValue.contents, rightValue.contents)}
        | _ ->
            let stateFromLeft = explode leftRef {state with Depth = state.Depth + 1}
            {explode rightRef stateFromLeft with Depth = state.Depth}

In [ ]:
type SplitState = { Split: int option }

let rec split (numberRef) state =
    match numberRef with
    | {contents = Literal value} ->
        if (value.Value < 10) then state
        else
            match state.Split with
            | Some _ -> state
            | None ->
                numberRef.Value <- Pair ((value.Value / 2) |> ref |> Literal |> ref, ((value.Value + 1) / 2) |> ref |> Literal |> ref)
                {Split = Some value.Value}
    | {contents = Pair (leftRef, rightRef)} ->
        state
        |> split leftRef
        |> split rightRef

In [ ]:
let rec magnitude (numberRef) =
    match numberRef with
    | {contents = Literal value} ->
        value.Value |> int64
    | {contents = Pair (leftRef, rightRef)} ->
        3L * magnitude leftRef + 2L * magnitude rightRef
        
let add number1Ref number2Ref =
    Pair (number1Ref, number2Ref)

In [ ]:
#load "../common.fsx"

let private reduction numberRef =
    let explosionState = {Depth = 0; Exploded = None; LiteralToLeft = None; LiteralToRight = None}
    let afterExplosionState = explode numberRef explosionState
    if afterExplosionState.Exploded.IsSome then Some()
    elif (split numberRef { Split = None }).Split.IsSome then Some()
    else None

let reduce numberRef =
    Seq.initInfinite (fun _ -> reduction numberRef)
    |> Seq.takeUntil (Option.isNone)
    |> Seq.last
    |> ignore
    numberRef

In [ ]:
#!value --name inputRaw --from-file ./data

In [ ]:
#!share inputRaw --from value
readLines inputRaw
|> Seq.map (parse >> ref)
|> Seq.reduce(fun n1 n2 -> add n1 n2 |> ref |> reduce)
|> magnitude

4132

In [ ]:
let private inputLines = readLines inputRaw

(inputLines, inputLines)
||> Seq.allPairs
|> Seq.collect (fun (n1,n2) -> [|(n1,n2); (n2,n1)|])
|> Seq.map (fun (n1, n2) -> add (parse n1 |> ref) (parse n2 |> ref) |> ref |> reduce)
|> Seq.map magnitude
|> Seq.max

4685